In [3]:
import os
import json
import requests
import random

In [4]:
import time

In [7]:
import os
from dotenv import load_dotenv

def configure_environment(dotenv_path: str = None) -> bool:
    """
    从 .env 文件加载环境变量。

    :param dotenv_path: .env 文件的路径。如果为 None，则从当前目录或父目录查找。
    :return: 如果成功加载文件则返回 True，否则返回 False。
    """
    # load_dotenv() 会查找 .env 文件并加载其内容到 os.environ
    # 如果找到了文件并加载成功，它会返回 True
    found = load_dotenv(dotenv_path)
    if not found:
        print("警告: 未找到 .env 文件。请确保该文件存在于项目根目录。")
    return found

## 从本地的env文件中读取环境变量

In [10]:
configure_environment()

True

In [12]:
class GeminiTranslator:
    """
    一个用于与 Google Gemini API 进行单次调用翻译的封装类。
    每次调用都是独立的，不维护对话历史。
    支持代理和自定义安全设置。
    """
    def __init__(self, api_key: str, model_name: str = "gemini-1.5-flash-latest", proxy: str = None):
        """
        初始化翻译器。

        :param api_key: 您的 Google Gemini API 密钥。
        :param model_name: 要使用的模型名称。
        :param proxy: (可选) 本地代理地址，例如 "socks5h://127.0.0.1:1080"。
        """
        if not api_key:
            raise ValueError("API key cannot be empty.")
        self.api_key = api_key
        self.model_name = model_name
        self.api_url = f"https://generativelanguage.googleapis.com/v1beta/models/{model_name}:generateContent"
        
        # 使用 Session 对象可以复用连接，提高性能
        self.session = requests.Session()

        if proxy:
            proxies = {"http": proxy, "https": proxy}
            self.session.proxies.update(proxies)
            print(f"--- [系统] 已配置代理: {proxy} ---")

        self.session.headers.update({
            "Content-Type": "application/json",
            "X-goog-api-key": self.api_key
        })
        
        # 安全配置，防止游戏内容被误拦
        self.safety_settings = [
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
        ]
        
        # 生成配置，使输出更稳定
        self.generation_config = {
            "temperature": 0.3,
            "topP": 1.0,
            "topK": 32,
            "maxOutputTokens": 8192,
        }
        self.translation_history = []

    def _build_single_prompt(self, task_prompt: str, glossary: dict, text_to_translate: str) -> str:
        """
        将任务指令、词汇表和待翻译内容合并成一个完整的单次请求Prompt。
        """
        # 1. 任务指令
        full_prompt = task_prompt
        
        # 2. 词汇表
        if glossary:
            full_prompt += "\n\n请严格遵守以下词汇对照表进行翻译，不要自行发挥：\n"
            full_prompt += "```json\n"
            full_prompt += json.dumps(glossary, indent=2, ensure_ascii=False)
            full_prompt += "\n```\n"

        # 3. 待翻译内容
        full_prompt += "\n以下是待翻译的内容：\n"
        full_prompt += "```\n"
        full_prompt += text_to_translate
        full_prompt += "\n```"
        
        return full_prompt

    def translate_single_shot(self, task_prompt: str, text_to_translate: str, glossary: dict = None) -> str | None:
        """
        在一次API调用中完成翻译任务。

        :param task_prompt: 描述翻译任务的初始提示语。
        :param text_to_translate: 需要翻译的英文文本。
        :param glossary: (可选) 一个英译中的词汇对照表字典。
        :return: 翻译后的中文文本，或在出错时返回 None。
        """
        # 构建包含所有信息的单个Prompt
        final_prompt = self._build_single_prompt(task_prompt, glossary, text_to_translate)
        
        # 构建API请求的payload
        payload = {
            "contents": [{"parts": [{"text": final_prompt}]}],
            "safetySettings": self.safety_settings,
            "generationConfig": self.generation_config,
        }

        print("--- [系统] 正在发送单次翻译请求... ---")
        # print("--- [调试] 发送的完整Prompt如下: ---\n", final_prompt) # 如果需要调试，可以取消此行注释
        
        try:
            response = self.session.post(self.api_url, json=payload, timeout=300)

            if not response.ok:
                print(f"--- [错误] 服务器返回状态码: {response.status_code} ---")
                try:
                    error_data = response.json()
                    print("服务器错误详情:", json.dumps(error_data, indent=2, ensure_ascii=False))
                except json.JSONDecodeError:
                    print("服务器原始响应:", response.text)
                return None

            response.raise_for_status()
            
            response_data = response.json()
            
            # 检查响应的有效性
            if 'candidates' in response_data and response_data['candidates']:
                candidate = response_data['candidates'][0]
                if 'finishReason' in candidate and candidate['finishReason'] in ['SAFETY', 'RECITATION']:
                    print(f"--- [错误] 模型生成的内容被阻止，原因: {candidate['finishReason']} ---")
                    print("安全评级详情:", candidate.get('safetyRatings'))
                    return None
                
                translated_text = candidate['content']['parts'][0]['text']
                self.translation_history.append(translated_text)
                return translated_text
            else:
                # 检查输入是否被阻止
                if 'promptFeedback' in response_data and 'blockReason' in response_data:
                    reason = response_data['promptFeedback']['blockReason']
                    print(f"--- [错误] 您的输入内容被安全策略阻止，原因: {reason} ---")
                else:
                    print("--- [警告] 模型未能提供有效翻译，响应中不包含'candidates'字段 ---")
                    print("完整回复:", response_data)
                return None
                
        except requests.exceptions.HTTPError as http_err:
            print(f"发生HTTP错误: {http_err}")
            return None
        except requests.exceptions.RequestException as e:
            print(f"发生网络错误: {e}")
            return None

## 初始化翻译器

In [15]:
# --- 1. 初始化 ---
# 强烈建议从环境变量获取API密钥
api_key = os.getenv("GEMINI_API_KEY")
if not api_key:
    print("请设置 GEMINI_API_KEY 环境变量！")
else:
    api_key = os.getenv("GEMINI_API_KEY")
    proxy_address = os.getenv("PROXY_URL") # 新增：获取代理地址
    
    translator = translator = GeminiTranslator(api_key=api_key,
                                               # proxy=proxy_address
                                              )
    print(f'翻译器初始化完成.')

翻译器初始化完成.


## prompt和词汇表配置

In [18]:
# 读取mod中的翻译对照词汇
en_gamestring = [i.strip() for i in open('../data/gamestring/GameString-enUS.txt').readlines()]
cn_gamestring = [i.strip() for i in open('../data/gamestring/GameString-zhCN.txt',encoding='utf-8').readlines()]
en_gs_map = {j[0]:j[1] for j in [i.split('=') for i in en_gamestring]}
cn_gs_map = {j[0]:j[1] for j in [i.split('=') for i in cn_gamestring]}

In [20]:
en_cn_vocab = {en_gs_map[k]:cn_gs_map[k] for k in cn_gs_map}

In [22]:
# --- 2. 定义翻译任务和词汇表 ---
# 定义初始Prompt，告诉AI它的角色和任务规则
my_task_prompt = """
你是一名专业的游戏汉化专家，正在为《星际争霸2》的一个MOD进行本地化工作。
你的任务是将我提供的英文文本翻译成简体中文。
请严格遵守以下规则：
1. 保持原始的数据格式 `key = value` 不变。
2. `value` 中可能包含格式控制字符，例如 `</n>`, `<c val="...">`, 等，请务必完整保留这些字符，不要做任何改动。
3. 翻译要符合《星际争霸2》的风格，术语要专业、统一。
4. 我会提供一个词汇对照表，请务必严格按照它来翻译，不要使用其他译名。
"""

# 定义你的专属词汇表，确保关键术语翻译统一
my_glossary = en_cn_vocab

## 测试翻译内容

In [25]:
def preprocess_gamestring_text(fpath:str):
    file_content = open(fpath,encoding='utf-8').read().replace('\ufeff','')
    return file_content

In [27]:
def postprocess_gamestring_text(gs_text:str):
    return gs_text.replace('```','')

In [29]:
# test_en_gs = preprocess_gamestring_text('../data/maps/thorner01.SC2Map/enUS.SC2Data/LocalizedData/GameStrings.txt')

# # --- 3. 开始一个新的翻译会话 ---
# response_trans_text = translator.translate_single_shot(task_prompt=my_task_prompt,
#                                  text_to_translate=test_en_gs,
#                                  glossary=my_glossary)

# response_trans_text = postprocess_gamestring_text(response_trans_text)

## 验证翻译内容和原始文本的键值对应

In [32]:
def raw_to_gs_map(raw_str):
    content_lst = [i for i in raw_str.split('\n') if not i == '']
    return {i[0]:i[1] for i in [i.split('=') for i in content_lst]}

# result_cn_gs_map = raw_to_gs_map(response_trans_text)

# origin_en_gs_map = raw_to_gs_map(test_en_gs)

In [34]:
# for key in origin_en_gs_map:
#     if not key in result_cn_gs_map:
#         print(key)

## 写出文件完成翻译

In [38]:
base_maps_path = '../data/maps/'
base_enus_folder = 'enUS.SC2Data'
base_zhcn_folder = 'zhCN.SC2Data'
local_folder = 'LocalizedData'
gs_fname = 'GameStrings.txt'

In [40]:
for map_name in os.listdir(base_maps_path):
    if not '.SC2Map' in map_name:
        continue
    print(f'Translation GameStrings for map:{map_name}.')
    gs_en_raw_text = preprocess_gamestring_text(os.path.join(base_maps_path, map_name, base_enus_folder, local_folder, gs_fname))

    translator = translator = GeminiTranslator(api_key=api_key,
                                                   # proxy=proxy_address
                                                  )
    gs_cn_raw_response = translator.translate_single_shot(task_prompt=my_task_prompt,
                                     text_to_translate=gs_en_raw_text,
                                     glossary=my_glossary)
    
    print(f'Translation finished, preparing write out.')
    gs_cn_raw_text = postprocess_gamestring_text(gs_cn_raw_response)
    
    # 验证key是否不存在
    result_cn_gs_map = raw_to_gs_map(response_trans_text)
    
    origin_en_gs_map = raw_to_gs_map(gs_en_raw_text)

    for key in origin_en_gs_map:
        if not key in result_cn_gs_map:
            print(f'In map {map_name}, key {key} not found in translation content.')

    # 写出文件
    cn_fpath = os.path.join(base_maps_path, map_name, base_zhcn_folder, local_folder, gs_fname)
    cn_gs_content = '\n'.join([f'{key}={result_cn_gs_map[key]}' for key in origin_en_gs_map if key in result_cn_gs_map])

    with open(cn_fpath,'w',encoding='utf-8') as file:
        file.write(cn_gs_content)
        
    waiting_second = random.randint(10,60)
    print(f'Successfully writing out for {map_name} then random waiting for:{waiting_second}second.')
    time.sleep(waiting_second)

Translation GameStrings for map:tarcade.SC2Map.
--- [系统] 正在发送单次翻译请求... ---
--- [错误] 服务器返回状态码: 503 ---
服务器错误详情: {
  "error": {
    "code": 503,
    "message": "The model is overloaded. Please try again later.",
    "status": "UNAVAILABLE"
  }
}
Translation finished, preparing write out.


AttributeError: 'NoneType' object has no attribute 'replace'